In [1]:
import warnings
import astropy.units as u
import ipyaladin.aladin_widget as ipal
import matplotlib.pyplot as plt
from style import (highlight_dec_cut, highlight_visibility, 
                   insert_tns_links_into_df,
                   highlight_gal_lat, get_styled_html_table,
                   highlight_general_traffic_light)
from tns_downloader import TNSDownloader
from visibility import insert_max_alt_in_frame, insert_gal_lat_in_frame
from visualization import get_aladin_coords
from IPython.display import (display, HTML, display_html,
                             Markdown, Latex)
from ipywidgets import Box, HBox, VBox, Label, Layout, Checkbox
from ipywidgets import HTML as HTML_widget
from astropy.coordinates import SkyCoord
from astropy.table import Table
from visibility import Visibility
from redshift import GladeRedshiftCatalogue
from functools import partial

In [2]:
rsc = GladeRedshiftCatalogue('redshift_data/glade_v2.3.hdf')

In [3]:
# Uncomment this to to parse data from the TNS

#tns_request = TNSDownloader(number_of_days=4)
#targets = insert_max_alt_in_frame(tns_request.result_compact)
#targets.to_hdf('data/targets.hdf', 'targets')
targets = pd.HDFStore('data/targets.hdf', mode='r')['targets']

# Classification targets

In [4]:
targets, names = insert_tns_links_into_df(targets)
targets = insert_gal_lat_in_frame(targets)

In [5]:
#HTML(get_styled_html_table(targets))

In [6]:
mask_alt = targets.max_alt > 30
mask_mag = targets['Discovery Mag'] > 17.
mask_classified = [
    isinstance(obj_type, str) for obj_type in targets['Obj. Type']
]
mask_unclassified = np.logical_not(mask_classified)
mask = np.logical_and(mask_alt, mask_mag)
#mask = np.logical_and(mask, mask_unclassified)
print(mask_alt.sum())
N_alt = np.logical_not(mask_alt).sum()
N_mag = np.logical_not(mask_mag).sum()

print('Discarding {} objects: {} based on mag, {} based on alt'.format(
    N_alt + N_mag, N_mag, N_alt)
)
targets = targets[mask]

52
Discarding 24 objects: 3 based on mag, 21 based on alt


In [7]:
HTML(get_styled_html_table(targets))

Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Discovering Group/s,Classifying Group/s,Disc. Internal Name,Discovery Mag,Discovery Mag Filter,Discovery Date (UT),max_alt,gal_lat
AT 2019lbj,11:30:43.630,-46:48:44.00,nan,nan,nan,nan,MASTER,nan,MASTER OT J113043.63-464844.0,18.2,Clear-,2019-07-10 19:35:06,51.2771,13.8425
AT 2019lbi,12:40:21.143,+01:16:26.45,nan,nan,nan,nan,ATLAS,nan,ATLAS19pbu,17.129,orange-ATLAS,2019-07-12 06:12:57,56.1861,64.0094
AT 2019lbg,23:39:35.606,-18:25:18.02,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dqy,21.78,w-PS1,2019-07-10 13:46:33,83.6851,-71.3208
AT 2019lbf,22:14:42.427,-10:22:40.31,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dqz,22.1,w-PS1,2019-07-10 12:34:33,75.6363,-49.5274
AT 2019lbe,23:18:32.710,-21:44:17.75,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dqw,20.21,w-PS1,2019-07-10 13:37:55,86.6809,-68.217
AT 2019lbd,23:23:41.609,-20:39:34.44,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dqx,20.94,w-PS1,2019-07-10 13:43:40,85.9166,-68.9621
AT 2019lbc,22:55:31.950,-13:35:43.56,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dqu,21.75,w-PS1,2019-07-10 12:44:38,78.8632,-59.7629
AT 2019lbb,22:45:20.491,-09:34:36.37,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dqv,19.91,w-PS1,2019-07-10 12:40:19,74.811,-55.4804
AT 2019lba,23:52:39.904,-18:44:11.01,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dqs,20.7,w-PS1,2019-07-10 13:47:59,83.9923,-74.0978
AT 2019laz,22:53:19.984,-12:02:02.62,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dqt,21.51,w-PS1,2019-07-10 12:41:45,77.2911,-58.4799


In [8]:
highlight_z = partial(highlight_general_traffic_light, levels=[0.03, 0.04, 0.12])
highlight_abs_mag = partial(highlight_general_traffic_light, levels=[-18.0, -17.5, 1e4])

In [11]:
%matplotlib widget
plt.ioff()

checks = []
aladins = []
tables = []

layout_aladin = Layout(width='40.00%')
layout_aladin.margin = '87.5px 0px 0px 75px'

layout = Layout(width='100%')
layout.margin = '100px px 0px 0px'

#widgets.append(HTML_widget('=' * 125))
for i in range(len(targets)):
    widgets = []
    
    label = Label(r'\(\color{red}{' + names.values[i] + '}\)')

    df_object = pd.DataFrame(targets.iloc[i].values[np.newaxis,:],
                             columns=targets.columns)

    aladin = ipal.Aladin(target=get_aladin_coords(targets, i),
                         layout=layout_aladin, fov=0.1,
                         survey='P/PanSTARRS/DR1/z') # 'P/SDSS9/color'
    
    # Visibility
    row = targets.iloc[i]
    coord = SkyCoord(row.RA, row.DEC, unit = (u.hourangle, u.deg))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        vis_fig = Visibility(skycoord=coord).plot()
    
    # Redshift
    m = rsc.find_host_z_from_row(row)
    
    if len(m) > 0:
        check = Checkbox(value=False, description='Classify me!',
                         disabled=False)
        widgets.append(HBox([label, check]))
        box = HBox([aladin, vis_fig.canvas], layout=layout)
        widgets.append(box)

        vbox = VBox(widgets, layout=Layout(width='100%'))

        display(vbox)

        checks.append(check)

        display(HTML(get_styled_html_table(df_object)))
        display(HTML(m[rsc.comp_repr].style.apply(
            highlight_z, subset=['z']).apply(highlight_abs_mag, subset=['abs_mag']
                                            ).hide_index().set_caption(
                                                    'Possible hosts').render()))
        table = rsc.generate_aladin_table(m)
        aladins.append(aladin)
        tables.append(table)
    else:
        continue

    
    display(HTML('<hr />' + '-' * 200))

# Add host galaxies to aladin widget
for aladin, table in zip(aladins, tables):
    aladin.add_table(table)

Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Discovering Group/s,Classifying Group/s,Disc. Internal Name,Discovery Mag,Discovery Mag Filter,Discovery Date (UT),max_alt,gal_lat
AT 2019lbi,12:40:21.143,+01:16:26.45,nan,nan,nan,nan,ATLAS,nan,ATLAS19pbu,17.129,orange-ATLAS,2019-07-12 06:12:57,56.1861,64.0094


z,flag1,d_proj[kpc],PGC,GWGC name,HyperLEDA name,2MASS name,SDSS-DR12 name,dist,dist_err,flag2,mu,abs_mag
0.0153342,G,4.85292,1.18981e+06,PGC1189811,PGC1189811,nan,nan,68.6869,nan,2,34.1835,-17.0545


Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Discovering Group/s,Classifying Group/s,Disc. Internal Name,Discovery Mag,Discovery Mag Filter,Discovery Date (UT),max_alt,gal_lat
AT 2019kzz,23:12:09.327,-20:36:55.39,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dpp,22.17,w-PS1,2019-07-10 13:40:47,85.8552,-66.4339


z,flag1,d_proj[kpc],PGC,GWGC name,HyperLEDA name,2MASS name,SDSS-DR12 name,dist,dist_err,flag2,mu,abs_mag
0.071487,G,9.2849,nan,nan,135390,23120906-2037008,nan,323.214,nan,1,37.6142,-15.4442


Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Discovering Group/s,Classifying Group/s,Disc. Internal Name,Discovery Mag,Discovery Mag Filter,Discovery Date (UT),max_alt,gal_lat
AT 2019kzw,23:10:36.873,-24:07:13.41,nan,nan,nan,nan,Pan-STARRS1,nan,PS19dpk,20.56,w-PS1,2019-07-10 13:39:21,89.3768,-67.1139


...

z,flag1,d_proj[kpc],PGC,GWGC name,HyperLEDA name,2MASS name,SDSS-DR12 name,dist,dist_err,flag2,mu,abs_mag
0.093007,G,5.22945,nan,nan,1254309,21395399+0332562,nan,426.988,nan,1,38.2175,-19.5625


Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Discovering Group/s,Classifying Group/s,Disc. Internal Name,Discovery Mag,Discovery Mag Filter,Discovery Date (UT),max_alt,gal_lat
SN 2019kvq,16:23:10.800,+00:48:38.59,SN Ia,0.058,nan,nan,"ZTF, GaiaAlerts",ZTF,ZTF19abetjcx,18.65,r-ZTF,2019-07-08 04:25:34,64.6018,32.8749


z,flag1,d_proj[kpc],PGC,GWGC name,HyperLEDA name,2MASS name,SDSS-DR12 name,dist,dist_err,flag2,mu,abs_mag
0.052731,G,8.77453,nan,nan,1176144,16231109+0048455,nan,234.201,nan,1,36.9248,-18.2748


Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Discovering Group/s,Classifying Group/s,Disc. Internal Name,Discovery Mag,Discovery Mag Filter,Discovery Date (UT),max_alt,gal_lat
AT 2019kuf,14:18:29.393,-31:39:43.83,nan,nan,nan,nan,ATLAS,nan,ATLAS19otn,18.942,orange-ATLAS,2019-07-08 06:44:38,82.734,27.657


z,flag1,d_proj[kpc],PGC,GWGC name,HyperLEDA name,2MASS name,SDSS-DR12 name,dist,dist_err,flag2,mu,abs_mag
0.045892,G,1.10749,nan,nan,702628,14182943-3139448,nan,205.936,nan,1,36.6126,-17.6706


## Generate finding charts, OBs

In [10]:
#pd.Series([check.value for check in checks], index=names.values)